In [944]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [945]:
# load the data

exercises_raw = pd.read_csv('data/megaGymDataset.csv')
exercies = exercises_raw['Title'].unique()

user_ratings_raw = pd.read_csv('data/user_rating_data.csv')
users = user_ratings_raw['User ID'].unique()

feature_type = exercises_raw['Type'].unique()

num_users = len(users)
num_exercises = len(exercies)
num_feature_type = len(feature_type)
num_recommmendations = 5



In [946]:
import sklearn
from sklearn import preprocessing

df = pd.read_csv('data/user_rating_data.csv')

r = df['Rating'].values.astype(np.float32)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(r.reshape(-1, 1))
df_normalized = pd.DataFrame(x_scaled)
df['Rating'] = df_normalized



In [947]:
import tensorflow as tf
import numpy as np

# Define the shape of the matrix
matrix = df.pivot(index='User ID', columns='Exercise', values='Rating').fillna(0)

matrix

Exercise,Cardio,Olympic Weightlifting,Plyometrics,Powerlifting,Strength,Stretching,Strongman
User ID,,,,,,,
0240de04-1ef3-4d1b-8ab1-09477185b5df,0.0,0.0,0.6,0.8,0.2,0.2,0.4
025cb182-d876-4574-9f55-036841f7222b,0.4,0.0,0.0,0.0,0.2,0.0,0.8
0383813b-386b-44c5-bb01-a3b801a0771a,0.2,0.4,1.0,0.6,1.0,0.6,0.4
05a3d754-f3a6-4013-8a2b-b0723579268f,0.2,0.8,0.0,0.0,0.0,0.0,0.0
08f05070-db91-41e5-ba59-5e85efd01431,0.2,0.6,0.0,0.8,0.0,0.2,0.0
...,...,...,...,...,...,...,...
f3682b5f-2037-488d-89a6-ba27fb4fa63b,0.4,0.0,0.0,0.0,0.0,0.0,0.0
f6b28a51-0891-452a-bf41-b492523030a4,0.0,0.0,1.0,0.6,0.0,0.6,0.0
f80c8104-12e5-4d81-8dab-140a50ceb1a5,0.2,0.2,0.0,0.4,0.6,0.0,0.4


In [948]:
users = matrix.index.tolist()
exercises = matrix.columns.tolist()

matrix = matrix.values


In [950]:
tf.compat.v1.disable_eager_execution()


num_input = num_exercises
num_hidden_1 = 10
num_hidden_2 = 5

X =tf.compat.v1.placeholder( tf.float32, [None, num_input] )    




wheights = {
    'encoder_h1': tf.Variable(tf.random.normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random.normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random.normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random.normal([num_hidden_1, num_input])),
}

biases = {
    'encoder_b1': tf.Variable(tf.random.normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random.normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random.normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random.normal([num_input])),
}

def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, wheights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, wheights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, wheights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, wheights['decoder_h2']), biases['decoder_b2']))
    return layer_2

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)



In [951]:

def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, wheights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, wheights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, wheights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, wheights['decoder_h2']), biases['decoder_b2']))
    return layer_2

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op

y_true = X


In [952]:
#loss function 

loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.compat.v1.train.RMSPropOptimizer(0.05).minimize(loss)

pred = pd.DataFrame()

# evaluation metrics

eval_x = tf.compat.v1.placeholder(tf.float32,)
eval_y = tf.compat.v1.placeholder(tf.float32, )

pre, pre_op = tf.compat.v1.metrics.precision(labels=eval_y, predictions=eval_x)






In [953]:
init = tf.compat.v1.global_variables_initializer()

local_init = tf.compat.v1.local_variables_initializer()

In [954]:
with tf.compat.v1.Session() as sess:

    
    epochs = 100
    batch_size = 50

    sess.run(init)
    sess.run(local_init)

    num_batches = int(matrix.shape[0]/batch_size)


    matrix =  np.array_split(matrix, num_batches)

    for i in range(epochs):
        avg_cost = 0
        for batch in matrix:
            print(batch.shape)

            _, l = sess.run([optimizer, loss], feed_dict={X: batch})
            print(_, l)
            avg_cost += l
            
        avg_cost = avg_cost / num_batches
        
        print("Epoch: {} cost={:.4f}".format(i+1, avg_cost))
    
    print("Predicting...")
    matrix = np.concatenate(matrix, axis=0)

    predictions = sess.run(decoder_op, feed_dict={X: matrix})
    pred = pred.append(pd.DataFrame(predictions))

    pred = pred.stack().reset_index(name='Rating')
    pred.columns = ['User ID', 'Exercise', 'Rating']
    pred['User ID'] = pred['User ID'].map(lambda value: users[value])
    pred['Exercise'] = pred['Exercise'].map(lambda value: exercises[value])

    


    


2023-02-27 21:03:47.880402: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-27 21:03:47.880444: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-02-27 21:03:48.884782: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-27 21:03:51.884518: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


(50, 7)


ValueError: Cannot feed value of shape (50, 7) for Tensor Placeholder_186:0, which has shape (None, 2909)